In [1]:
# import Librarairs
import os,json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import numpy as np
from sklearn.metrics import matthews_corrcoef
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

CWD = os.getcwd()  # get current directory path
BUG_REPORTS_DIR = os.path.join(CWD,"bugs")   # current directory+bugs
BUG_REPORTS_LIST = os.listdir(BUG_REPORTS_DIR)  # all the items in 'current_directory+bugs'

SENTI_REPORTS_DIR = os.path.join(CWD,"Sentiments_output")
SENTI_REPORTS_LIST = os.listdir(SENTI_REPORTS_DIR)



In [2]:
# Load data set for bugs and sentiments

print(BUG_REPORTS_DIR)
print('-----------------')
print(BUG_REPORTS_LIST)
print('-----------------')
print(SENTI_REPORTS_DIR)
print('-----------------')
print(SENTI_REPORTS_LIST)     

/home/cr/Desktop/bugs
-----------------
['Core.json', 'Toolkit.json', 'bugzilla.json', 'Calendar.json', 'SeaMonkey.json', 'Camino Graveyard.json', 'MailNews Core.json', 'Firefox.json', 'Thunderbird.json', 'Core Graveyard.json']
-----------------
/home/cr/Desktop/Sentiments_output
-----------------
['Camino_Graveyard_sentiments.csv', 'bugzilla_sentiments.csv', 'Firefox_sentiments.csv', 'Core_sentiments.csv', 'MailNews_Core_sentiments.csv', 'SeaMonkey_sentiments.csv', 'Thunderbird_sentiments.csv', 'Toolkit_sentiments.csv', 'Core_Graveyard_sentiments.csv', 'Calendar_sentiments.csv']


In [67]:
# Extract data from files for sentiments
import csv
test_senti = []
train_senti=[]
import pandas as pd
for count, report in enumerate(SENTI_REPORTS_LIST):
    
    if count == 6:
        #add this project to test set
        print(f'Test Set: {report}')
        with open(os.path.join(SENTI_REPORTS_DIR,report), 'r') as report_file:
            temp_data = pd.read_csv(report_file)
            print(len(temp_data))
            #print(temp_data['neutral'])
            test_senti.extend(temp_data.iloc[0:, 1])
            
    else:
        #appent this project to train set
        print(f'Train Set: {report}')
        with open(os.path.join(SENTI_REPORTS_DIR,report), 'r',encoding='cp850') as report_file:
            temp_data = pd.read_csv(report_file)
            print(len(temp_data))
            #print(temp_data.iloc[0:, 1])
            train_senti.extend(temp_data.iloc[0:, 1])
            
            #print(f"File:{report} <> Reports: {len(temp_data['bugs'])}")



Train Set: Camino_Graveyard_sentiments.csv
1291
Train Set: bugzilla_sentiments.csv
5325
Train Set: Firefox_sentiments.csv
7369
Train Set: Core_sentiments.csv
7229
Train Set: MailNews_Core_sentiments.csv
2797
Train Set: SeaMonkey_sentiments.csv
8717
Test Set: Thunderbird_sentiments.csv
8717
Train Set: Toolkit_sentiments.csv
1900
Train Set: Core_Graveyard_sentiments.csv
889
Train Set: Calendar_sentiments.csv
1786


In [92]:
# Load data for bugs 
train_set = []
test_set = []

for count, report in enumerate(BUG_REPORTS_LIST):
    if count == 9:
        #add this project to test set
        print(f'Test Set: {report}')
        with open(os.path.join(BUG_REPORTS_DIR,report), 'r',encoding='cp850') as report_file:
            temp_data = json.load(report_file)
            print(f"File:{report} <> Reports: {len(temp_data['bugs'])}")
            test_set.extend(temp_data["bugs"])
    else:
        #appent this project to train set
        print(f'Train Set: {report}')
        with open(os.path.join(BUG_REPORTS_DIR,report), 'r',encoding='cp850') as report_file:
            temp_data = json.load(report_file)
            print(f"File:{report} <> Reports: {len(temp_data['bugs'])}")
            train_set.extend(temp_data["bugs"])   

Train Set: Core.json
File:Core.json <> Reports: 7228
Train Set: Toolkit.json
File:Toolkit.json <> Reports: 1899
Train Set: bugzilla.json
File:bugzilla.json <> Reports: 5324
Train Set: Calendar.json
File:Calendar.json <> Reports: 1785
Train Set: SeaMonkey.json
File:SeaMonkey.json <> Reports: 8716
Train Set: Camino Graveyard.json
File:Camino Graveyard.json <> Reports: 1290
Train Set: MailNews Core.json
File:MailNews Core.json <> Reports: 2796
Train Set: Firefox.json
File:Firefox.json <> Reports: 7368
Train Set: Thunderbird.json
File:Thunderbird.json <> Reports: 5129
Test Set: Core Graveyard.json
File:Core Graveyard.json <> Reports: 888


In [93]:
print(f'Train len: {len(train_set)}')
print(f'Test len: {len(test_set)}')
print(f'Total len: {len(train_set)+len(test_set)}')

Train len: 41535
Test len: 888
Total len: 42423


In [94]:
summary_train=[]
label_train=[]
summary_test=[]
label_test=[]
for i in train_set:
    summary_train.append(i['summary'])
    #summary_train=summary_train[0:10]
    label_train.append(i['resolution'])
print('train_summary',len(summary_train))
print('train_label',len(label_train))

    #label_train=label_train[0:10]
for j in test_set:
    summary_test.append(j['summary'])
    #summary_test=summary_test[0:10]
    label_test.append(j['resolution'])
   # label_test=label_test[0:10]
print('test_summary',len(summary_test))
print('test_label',len(label_test))

train_summary 41535
train_label 41535
test_summary 888
test_label 888


In [95]:
def senti_to_int(train_test_senti):
    
    train_test_sentiments=[]
    for x in range(len(train_test_senti)):
        if train_test_senti[x]=="positive":
            train_test_senti[x]=1
        elif train_test_senti[x]=="neutral":
            train_test_senti[x]=1
        elif train_senti[x]=="negative":
            train_test_senti[x]=0
        train_test_sentiments.append(train_test_senti[x])
    return train_test_sentiments


In [96]:
train_sentiment=senti_to_int(train_senti)
test_sentiment=senti_to_int(test_senti)
print(f"Training sentimetns Length:{len(train_sentiment)}")
print(f"Testing sentiments Length:{len(test_sentiment)}")

Training sentimetns Length:37303
Testing sentiments Length:8717


In [97]:
# Sentiments 

train_senti=np.array(train_sentiment)
test_senti=np.array(test_sentiment)
#train_sentiments.shape
train_sentiments=train_senti.reshape(len(train_senti),1)
test_sentiments=test_senti.reshape(len(test_senti),1)


##  use 
# train_sentiments
# test_sentiments

In [98]:
print(f'Train len: {len(train_sentiments)}')
print(f'Test len: {len(test_sentiments)}')
print(f'Total len: {len(train_sentiments)+len(test_sentiments)}')
print(test_sentiments.shape)
print(train_sentiments.shape)

Train len: 37303
Test len: 8717
Total len: 46020
(8717, 1)
(37303, 1)


In [99]:
#Preprocess sentences
import re
def train_test_summary(summaries_text):
    
    lemmas=[]
    lemmatized_word=[]
    train_test_summaries=[]
    for count, summaries in enumerate (summaries_text):
        text = re.sub(r'[^a-zA-Z\']', ' ', summaries)
        #removes all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
        #substitute multiple spcaces with single space
        s_m_s = re.sub(r'\s+', ' ', document, flags=re.I)
        #removes prefix
        r_prefix = re.sub(r'^b\s+', '', s_m_s)
        #lemmas.append(r_prefix)
        r_pm = re.sub(r'[^\w\s]', '', r_prefix)
        step1 = r_pm.replace("`` ", '"').replace(" ''", '"').replace('. . .',  '...')
        step2 = step1.replace(" ( ", " (").replace(" ) ", ") ")
        step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
        step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
        step5 = step4.replace(" '", "'").replace(" n't", "n't").replace(
        "can not", "cannot")
        step6 = step5.replace(" ` ", " '")
        step7=step6.strip()
        step8=step7.lower()    
        lemmas=[]
        sentences = nltk.word_tokenize(step7)
        lemmatizer = WordNetLemmatizer()
        stem_words=[]
        for sentence_words in sentences:
            lower=sentence_words.lower()
            if lower not in stopwords.words('english'):
                lemma = lemmatizer.lemmatize(lower)
                stem_words.append(lemma)
            untokenized_sentence= ' '.join(stem_words)
        train_test_summaries.append(untokenized_sentence)
    return train_test_summaries

In [100]:
# Encocede Labels 
def train_test_labels(labels):
    train_test_label=[]
    for x in range(len(labels)):
        
        
        if label_train[x]=="FIXED":
            label_train[x]=1
        elif label_train[x]=="WONTFIX":
            label_train[x]=0
        elif label_train[x]=="INVALID":
            label_train[x]=0
        elif label_train[x]=="INCOMPLETE":
            label_train[x]=0
        elif label_train[x]=="WONTFIX":
            label_train[x]=0
        elif label_train[x]=="WORKSFORME":
            label_train[x]=0
        elif label_train[x]=="EXPIRED":
            label_train[x]=0
        elif label_train[x]=="DUPLICATE":
            label_train[x]=0
        elif label_train[x]=="":
            label_train[x]=0
        train_test_label.append(label_train[x])
        
    
    
    #print('train_lbel', len(train_label))
    return train_test_label
    


In [101]:
train_labels= train_test_labels(label_train)
test_labels=train_test_labels(label_test)
print(f"Training Labels Length:{len(train_labels)}")
print(f"Testing Labels Length:{len(test_labels)}")


# Use 
#     train_labels
#     test_labels

Training Labels Length:41535
Testing Labels Length:888


In [102]:

train_summary=train_test_summary(summary_train)
test_summary=train_test_summary(summary_test)

print(f"Training summary Length:{len(train_summary)}")
print(f"Testing summary Length:{len(test_summary)}")



# Use 
#     train_summary
#     test_summary

Training summary Length:41535
Testing summary Length:888


In [103]:
print(train_summary[0])

feature cs first letter selector isnt finished


In [104]:
#convert Labels (integers) to numpy array for feeding to a model and for evaluation

train_labels=np.array(train_labels)
test_labels=np.array(test_labels)

In [105]:
#import Libraries
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
#from gensim.models import doc2vec
from collections import namedtuple
import pandas as pd

In [106]:
#BOW

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(max_features=3000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X_train_counts = count_vect.fit_transform(train_summary)
X_train_counts.shape


# here shape shows rows and colums, colums are according to feature number, total no.of features=total no. of columns

(41535, 3000)

In [107]:
vec = CountVectorizer().fit(train_summary)
bag_of_words = vec.transform(train_summary)
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in     vec.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

print(words_freq[:10])

[('add', 3894), ('tab', 3142), ('rfe', 2712), ('message', 2695), ('window', 2581), ('new', 2522), ('option', 2397), ('mail', 2357), ('page', 1966), ('folder', 1942)]


In [108]:
b=vec.vocabulary_.get(u'should')
print(b)

#The index value of a word in the vocabulary is linked to its frequency in the whole training corpus.

None


In [109]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(41535, 3000)

In [110]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(41535, 3000)

In [111]:
X_new_counts = count_vect.transform(test_summary)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

In [112]:
#MUltinomail_naive Bayes

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, train_labels)
predicted = clf.predict(X_new_tfidf)
print(metrics.classification_report(test_labels, predicted))

              precision    recall  f1-score   support

           0       0.68      0.96      0.80       606
           1       0.35      0.05      0.09       282

    accuracy                           0.67       888
   macro avg       0.52      0.50      0.44       888
weighted avg       0.58      0.67      0.57       888



In [409]:
#SVM
from sklearn import svm
clf = svm.SVC()
svm_clf=clf.fit(X_train_tfidf, train_label)
SVC()
predictions=svm_clf.predict(X_new_tfidf)
print(metrics.classification_report(test_label, predictions))

/home/cr/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


              precision    recall  f1-score   support

           0       0.66      1.00      0.79      4754
           1       0.00      0.00      0.00      2474

    accuracy                           0.66      7228
   macro avg       0.33      0.50      0.40      7228
weighted avg       0.43      0.66      0.52      7228



/home/cr/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [415]:
#Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
decision_tree = decision_tree.fit(X_train_tfidf, train_label)
prediction=decision_tree.predict(X_new_tfidf)
print(metrics.classification_report(test_label, prediction))

              precision    recall  f1-score   support

           0       0.66      1.00      0.79      4754
           1       1.00      0.00      0.00      2474

    accuracy                           0.66      7228
   macro avg       0.83      0.50      0.40      7228
weighted avg       0.77      0.66      0.52      7228



In [416]:
#Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train_tfidf, train_label) 
y_pred = classifier.predict(X_new_tfidf)
print(metrics.classification_report(test_label, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.97      0.79      4754
           1       0.40      0.03      0.06      2474

    accuracy                           0.65      7228
   macro avg       0.53      0.50      0.42      7228
weighted avg       0.57      0.65      0.54      7228



In [533]:
#Term frequency = (Number of Occurrences of a word)/(Total words in the document)
#IDF(word) = Log((Total number of documents)/(Number of documents containing the word))


# There are two ways to create TF_IDF embeddings 
# 1. make vecrors first using COuntVectorizer to generate one hot encoding vectors BOW (1,0 if exist,then 1. otherwise 0)
# and then transform it into TF-IDF using TF-IDF transformer 
# 2. Directly use TF-IDF vectorizer to generate TF-IDF embeddings to feed a model 

# we can use here uni_gram, bi-Gram and tri_gram embedding also, then it will generate differnt embeddings.

# 1st method
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=10000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(train_summary).toarray()

# till here it is simple embedding into 0 if words doesn't exist, and n=no.of times a word occurs
# documents: #1. ali go go to school, #2. go to shooingmall, #3. takes rest , #4. go to sleeop
# features set: 1. ali, 2. go, 3. school, 4. shoppingmall, 5. take, 6. rest, 7. sleep
# embedding dimensions(3,7)  # Rows = no.of documents & columns = no.of features   
# embedding for document #1 = [1,2,1,0,0,0,0]


from sklearn.feature_extraction.text import TfidfTransformer
TfidfTransformer = TfidfTransformer()
X_train= TfidfTransformer.fit_transform(X).toarray()
X_test=vectorizer.transform(test_summary).toarray()

In [534]:
#In order to make the vectorizer => transformer => classifier easier to work with, 
#scikit-learn provides a Pipeline class that behaves like a compound classifier:

from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
text_clf = Pipeline([
('vect', CountVectorizer()),
('tfidf', TfidfTransformer()),
('clf', MultinomialNB()),
])

TypeError: 'TfidfTransformer' object is not callable

In [774]:
#text_clf.fit(X_train, train_label)

In [511]:
# MNB
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train, train_label)
predicted = clf.predict(X_test)
print(metrics.classification_report(test_label, predicted))

              precision    recall  f1-score   support

           0       0.67      0.85      0.75      4754
           1       0.41      0.20      0.27      2474

    accuracy                           0.63      7228
   macro avg       0.54      0.53      0.51      7228
weighted avg       0.58      0.63      0.59      7228



In [512]:
#Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
decision_tree = decision_tree.fit(X_train, train_label)
prediction=decision_tree.predict(X_test)
print(metrics.classification_report(test_label, prediction))

              precision    recall  f1-score   support

           0       0.66      1.00      0.79      4754
           1       1.00      0.00      0.00      2474

    accuracy                           0.66      7228
   macro avg       0.83      0.50      0.40      7228
weighted avg       0.77      0.66      0.52      7228



In [32]:
                            # just BOW Approach # Used By QASIM

    
    
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=2000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(train_summary).toarray()
Y=vectorizer.transform(test_summary).toarray()
X=np.array(X)

In [33]:
# Senitments Appended

senti_summary_train= np.append(X,train_sentiments, axis=1)
senti_summary_test= np.append(Y,test_sentiments, axis=1)
X1=senti_summary_train
Y1=senti_summary_test

In [64]:
# 1. MNB
from sklearn.naive_bayes import MultinomialNB
import scipy.stats as stats
clf = MultinomialNB().fit(X,train_labels)
predicted = clf.predict(Y)
print(metrics.classification_report(test_labels,predicted))


roc=roc_auc_score(test_labels,predicted)
print('roc',roc)
metheows=matthews_corrcoef(test_labels,predicted)
print('MATHEOS_COEFFICIENT',metheows)
odds_ratio=np.exp(clf.coef_)
print('ODDS RATIO',odds_ratio)
matrix = metrics.confusion_matrix(test_labels,predicted)
print('CONFUSION MATRIX',matrix)

              precision    recall  f1-score   support

           0       0.67      0.84      0.75      4754
           1       0.41      0.22      0.29      2474

    accuracy                           0.63      7228
   macro avg       0.54      0.53      0.52      7228
weighted avg       0.58      0.63      0.59      7228

roc 0.5285217843188003
MATHEOS_COEFFICIENT 0.06997114109487436
ODDS RATIO [[5.50372878e-05 3.16464405e-03 1.81623050e-03 ... 5.50372878e-05
  7.70522029e-04 8.25559316e-05]]
CONFUSION MATRIX [[3976  778]
 [1928  546]]


In [66]:
# 2.  Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
decision_tree = decision_tree.fit(X, train_labels)
prediction=decision_tree.predict(Y)
print(metrics.classification_report(test_labels, prediction))




roc=roc_auc_score(test_labels, prediction)
print('roc',roc)
metheows=matthews_corrcoef(test_labels, prediction)
print('MATHEOS_COEFFICIENT',metheows)
odds_ratio=np.exp(clf.coef_)
print('ODDS RATIO',odds_ratio)
matrix = metrics.confusion_matrix(test_labels, prediction)
print('CONFUSION MATRIX',matrix)


              precision    recall  f1-score   support

           0       0.66      1.00      0.79      4754
           1       1.00      0.00      0.00      2474

    accuracy                           0.66      7228
   macro avg       0.83      0.50      0.40      7228
weighted avg       0.77      0.66      0.52      7228

roc 0.5002021018593371
MATHEOS_COEFFICIENT 0.016306122989834953
ODDS RATIO [[5.50372878e-05 3.16464405e-03 1.81623050e-03 ... 5.50372878e-05
  7.70522029e-04 8.25559316e-05]]
CONFUSION MATRIX [[4754    0]
 [2473    1]]


In [69]:
# 3. Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X, train_labels) 
y_pred = classifier.predict(Y)
print(metrics.classification_report(test_labels, y_pred))

roc=roc_auc_score(test_labels, y_pred)
print('roc',roc)
metheows=matthews_corrcoef(test_labels, y_pred)
print('MATHEOS_COEFFICIENT',metheows)
odds_ratio=np.exp(clf.coef_)
print('ODDS RATIO',odds_ratio)
matrix = metrics.confusion_matrix(test_labels, y_pred)
print('CONFUSION MATRIX',matrix)

              precision    recall  f1-score   support

           0       0.66      0.94      0.78      4754
           1       0.43      0.09      0.15      2474

    accuracy                           0.65      7228
   macro avg       0.55      0.51      0.46      7228
weighted avg       0.58      0.65      0.56      7228

roc 0.5139617780066245
MATHEOS_COEFFICIENT 0.05173626501844653
ODDS RATIO [[5.50372878e-05 3.16464405e-03 1.81623050e-03 ... 5.50372878e-05
  7.70522029e-04 8.25559316e-05]]
CONFUSION MATRIX [[4464  290]
 [2254  220]]


In [73]:
# 4. Logistic Regression 

from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression(random_state=0).fit(X, train_labels)
predicted = clf_lr.predict(Y)
print(metrics.classification_report(test_labels, predicted))

roc=roc_auc_score(test_labels, predicted)
print('roc',roc)
metheows=matthews_corrcoef(test_labels, predicted)
print('MATHEOS_COEFFICIENT',metheows)
odds_ratio=np.exp(clf.coef_)
print('ODDS RATIO',odds_ratio)
matrix = metrics.confusion_matrix(test_labels, predicted)
print('CONFUSION MATRIX',matrix)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

           0       0.66      0.95      0.78      4754
           1       0.38      0.06      0.11      2474

    accuracy                           0.64      7228
   macro avg       0.52      0.51      0.44      7228
weighted avg       0.57      0.64      0.55      7228

roc 0.5053229225510305
MATHEOS_COEFFICIENT 0.021811825836576235
ODDS RATIO [[5.50372878e-05 3.16464405e-03 1.81623050e-03 ... 5.50372878e-05
  7.70522029e-04 8.25559316e-05]]
CONFUSION MATRIX [[4501  253]
 [2316  158]]


In [75]:
# 5. Bernouli Naive Bayes
from sklearn.naive_bayes import BernoulliNB
clf_bn = BernoulliNB()
clf_bn.fit(X,train_labels)
predict=clf_bn.predict(Y)
print(metrics.classification_report(test_labels, predict))

roc=roc_auc_score(test_labels, predict)
print('roc',roc)
metheows=matthews_corrcoef(test_labels, predict)
print('MATHEOS_COEFFICIENT',metheows)
odds_ratio=np.exp(clf.coef_)
print('ODDS RATIO',odds_ratio)
matrix = metrics.confusion_matrix(test_labels, predict)
print('CONFUSION MATRIX',matrix)

              precision    recall  f1-score   support

           0       0.67      0.85      0.75      4754
           1       0.41      0.20      0.27      2474

    accuracy                           0.63      7228
   macro avg       0.54      0.53      0.51      7228
weighted avg       0.58      0.63      0.59      7228

roc 0.5267089893070517
MATHEOS_COEFFICIENT 0.06809179559983118
ODDS RATIO [[5.50372878e-05 3.16464405e-03 1.81623050e-03 ... 5.50372878e-05
  7.70522029e-04 8.25559316e-05]]
CONFUSION MATRIX [[4051  703]
 [1976  498]]


In [80]:
# 6. MLP 
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5 , hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X,train_labels)
MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,solver='lbfgs')
predcitons=clf.predict(Y)
print(metrics.classification_report(test_labels, predcitons))

roc=roc_auc_score(test_labels, predcitons)
print('roc',roc)
metheows=matthews_corrcoef(test_labels, predcitons)
print('MATHEOS_COEFFICIENT',metheows)



              precision    recall  f1-score   support

           0       0.67      0.82      0.74      4754
           1       0.39      0.21      0.27      2474

    accuracy                           0.61      7228
   macro avg       0.53      0.52      0.51      7228
weighted avg       0.57      0.61      0.58      7228

roc 0.5183713736022492
MATHEOS_COEFFICIENT 0.04454252047934634


AttributeError: 'MLPClassifier' object has no attribute 'coef_'

In [81]:
matrix = metrics.confusion_matrix(test_labels, predcitons)
print('CONFUSION MATRIX',matrix)
odds_ratio=np.exp(clf.coef_)
print('ODDS RATIO',odds_ratio)

CONFUSION MATRIX [[3916  838]
 [1947  527]]


AttributeError: 'MLPClassifier' object has no attribute 'coef_'

In [82]:
# 07. KNN
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()  
model.fit(X,train_labels)

probs = model.predict_proba(Y)  
probs = probs[:, 1]  
fper, tper, thresholds = roc_curve(test_labels, probs) 
plot_roc_curve(fper, tper)
print(metrics.classification_report(test_labels, probs))
roc=roc_auc_score(test_labels, probs)
print('roc',roc)
metheows=matthews_corrcoef(test_labels, probs)
print('MATHEOS_COEFFICIENT',metheows)
odds_ratio=np.exp(clf.coef_)
print('ODDS RATIO',odds_ratio)
matrix = metrics.confusion_matrix(test_labels, probs)
print('CONFUSION MATRIX',matrix)

NameError: name 'plot_roc_curve' is not defined

In [ ]:
#7. SVM
from sklearn import svm
clf = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
svm_clf=clf.fit(X, train_labels)
SVC()
predictions=svm_clf.predict(Y)
print(metrics.classification_report(test_labels, predictions))


roc=roc_auc_score(test_labels, predictions)
print('roc',roc)
metheows=matthews_corrcoef(test_labels, predictions)
print('MATHEOS_COEFFICIENT',metheows)
odds_ratio=np.exp(clf.coef_)
print('ODDS RATIO',odds_ratio)
matrix = metrics.confusion_matrix(test_labels, predictions)
print('CONFUSION MATRIX',matrix)

/home/cr/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [113]:
                                # TF-IDF used by Nizamani 
# 2nd method

from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = tfidfconverter.fit_transform(train_summary).toarray()
Y=tfidfconverter.transform(test_summary).toarray()




# max_features=no. of max_time_occuring_features, you want to use to feed into the model
# min_df= minimimum document frequency, select the word that occurs in atleat 5 documents(summaris)
# max_df= maximum document frequency, select the word that occurs not more than 70% of the documents 

In [120]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

count_vect= TfidfVectorizer()
X_train_counts=count_vect.fit_transform(train_summary)
X_new_counts=count_vect.transform(test_summary)
X_new_tfidf=tfidf_transformer.transform(X_new_counts)

ValueError: Input has n_features=13549 while the model has been trained with n_features=3000

In [ ]:
# 1. MNB 
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X, train_labels)
predicted = clf.predict(Y)
print(metrics.classification_report(test_labels, predicted)

In [115]:
# 1. MNB 
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X, train_labels)
predicted = clf.predict(Y)
print(metrics.classification_report(test_labels, predicted))

              precision    recall  f1-score   support

           0       0.69      0.97      0.80       606
           1       0.38      0.05      0.08       282

    accuracy                           0.67       888
   macro avg       0.53      0.51      0.44       888
weighted avg       0.59      0.67      0.57       888



In [ ]:
# 1. MNB 
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X, train_labels)
predicted = clf.predict(Y)
print(metrics.classification_report(test_labels, predicted))

In [ ]:
# 2.  Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
decision_tree = decision_tree.fit(X, train_label)
prediction=decision_tree.predict(Y)
print(metrics.classification_report(test_label, prediction))

In [ ]:
# 3. Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X, train_label) 
y_pred = classifier.predict(Y)
print(metrics.classification_report(test_label, y_pred))

In [ ]:
# 4. Logistic Regression 

from sklearn.linear_model import LogisticRegression
clf_lr = LogisticRegression(random_state=0).fit(X, train_label)
predicted = clf_lr.predict(Y)
print(metrics.classification_report(test_label, predicted))

In [ ]:
# 5. Bernouli Naive Bayes
from sklearn.naive_bayes import BernoulliNB
clf_bn = BernoulliNB()
clf_bn.fit(X, train_label)
predict=clf_bn.predict(Y)
print(metrics.classification_report(test_label, predict))

In [ ]:
#6. SVM
from sklearn import svm
clf = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
svm_clf=clf.fit(X, train_label)
SVC()
predictions=svm_clf.predict(Y)
print(metrics.classification_report(test_label, predictions))

/home/cr/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [ ]:

#6 again SVM
from sklearn import model_selection, naive_bayes, svm
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X, train_label)
predictions=SVM.predict(Y)
print(metrics.classification_report(test_label, predictions))

In [556]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_summary)
X_train_counts.shape
count_vect.vocabulary_.get(u'algorithm')

#The index value of a word in the vocabulary is linked to its frequency in the whole training corpus.

268

In [562]:
print(X_train_counts[0])

  (0, 1989)	1
  (0, 4387)	1
  (0, 6676)	1
  (0, 7952)	1
  (0, 8150)	1
  (0, 8461)	1
  (0, 10196)	1


In [545]:

print(len(train_summary))

35195


In [574]:
# From occurrences to frequencies¶
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(35195, 10588)

In [575]:
print(X_train_tf[0])

  (0, 1989)	0.3779644730092272
  (0, 4387)	0.3779644730092272
  (0, 6676)	0.3779644730092272
  (0, 7952)	0.3779644730092272
  (0, 8150)	0.3779644730092272
  (0, 8461)	0.3779644730092272
  (0, 10196)	0.3779644730092272


In [576]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(35195, 10588)

In [577]:
print(X_train_tfidf[0])

  (0, 10196)	0.49374309343903616
  (0, 8461)	0.536619147489694
  (0, 8150)	0.3389371690811846
  (0, 7952)	0.20724042931785985
  (0, 6676)	0.2908051104736087
  (0, 4387)	0.28607161155816424
  (0, 1989)	0.3795076664350987


In [581]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', MultinomialNB()),
 ])

In [584]:
# text_clf.fit(X_train_tfidf,train_labels)
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(test_labels, prediction, pos_label=2)

In [ ]:
#doc_clf.predict(tf_idf.transform((count_vect.transform([r'document']))))